In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

end = datetime.now()
start = datetime(end.year-10,end.month,end.day)

exchange_pair = "BTC-USD"
bitcoin_data = yf.download(exchange_pair, start, end)

bitcoin_data.head(10)

In [ ]:
bitcoin_data.describe()

In [ ]:
bitcoin_data.info()

In [ ]:
bitcoin_data.isna().sum()

In [ ]:
Closing_price = bitcoin_data[['Close']]
Closing_price

In [ ]:
plt.figure(figsize=(16,8))
Closing_price['Close'].plot()
plt.xlabel('Years')
plt.ylabel('Close Price')
plt.title('Close price of Bitcoin')

In [ ]:
for i in range(2014,2024):
    print(list(Closing_price.index.year).count(i))

In [ ]:
Closing_price['MA_for_365_days'] = Closing_price['Close'].rolling(365).mean()
Closing_price['MA_for_365_days'].head()

In [ ]:
Closing_price['MA_for_365_days'][0:366].tail()

In [ ]:
plt.figure()
Closing_price[['Close', 'MA_for_365_days']].plot(figsize=(16,8))
plt.xlabel('Years')
plt.ylabel('Close Price')
plt.title('Close price of bitcoin')

In [ ]:
Closing_price['MA_for_100_days'] = Closing_price['Close'].rolling(100).mean()
Closing_price['MA_for_100_days'].head()
plt.figure()
Closing_price[['Close', 'MA_for_100_days']].plot(figsize=(16,8))
plt.xlabel('Years')
plt.ylabel('Close Price')
plt.title('Close price of bitcoin')

In [ ]:
Closing_price['MA_for_20_days'] = Closing_price['Close'].rolling(20).mean()
Closing_price['MA_for_20_days'].head()
plt.figure()
Closing_price[['Close', 'MA_for_20_days']].plot(figsize=(16,8))
plt.xlabel('Years')
plt.ylabel('Close Price')
plt.title('Close price of bitcoin')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(Closing_price[['Close']].values)
scaled_data

In [ ]:
scaled_data.shape

In [ ]:
x_data = []
y_data = []

base_days = 100

for i in range(base_days,len(scaled_data)):
    x_data.append(scaled_data[i-base_days:i])
    y_data.append(scaled_data[i])

x_data, y_data = np.array(x_data), np.array(y_data)


x_data[0], y_data[0]

In [ ]:
len_train = int(len(x_data)*0.9)
x_train = x_data[:len_train]
y_train = y_data[:len_train]

x_test = x_data[len_train:]
y_test = y_data[len_train:]

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()

model.add(LSTM(128, return_sequences = True, input_shape = (x_train.shape[1],1)))

model.add(LSTM(64, return_sequences = False))

model.add(Dense(25))

model.add(Dense(1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, batch_size = 5, epochs=10)

In [ ]:
predictions = model.predict(x_test)
predictions

In [ ]:
inv_predictions = scaler.inverse_transform(predictions)
inv_predictions

In [ ]:
inv_y_test = scaler.inverse_transform(y_test)

In [ ]:
inv_predictions[:5], inv_y_test[:5]

In [ ]:
plotting_data = pd.DataFrame({
  'original_test_data': inv_y_test.reshape(-1),
'predictions': inv_predictions.reshape(-1),
},
index = Closing_price.index[len_train+100:]
)

plotting_data.head()

In [ ]:
plt.figure()
plotting_data.plot(figsize=(15,6))
plt.xlabel('Years')
plt.ylabel('Close Price')
plt.title('Close price of bitcoin')

In [ ]:
last_100 = Closing_price[['Close']].tail(100)
last_100.tail()

In [ ]:
last_100 = scaler.fit_transform(last_100['Close'].values.reshape(-1,1)).reshape(1,-1,1)
last_100

In [ ]:
last_100.shape

In [ ]:
day101 = model.predict(last_100)
day101

In [ ]:
scaler.inverse_transform(day101)

In [ ]:
def predict_future(no_of_days, prev_100):
    future_predictions = []
    
    for i in range(no_of_days):
        next_day = model.predict(prev_100).tolist()
        prev_100_list = prev_100[0].tolist()
        prev_100_list.append(next_day[0])
        prev_100 = np.array([prev_100_list[1:]])
        
        future_predictions.append(scaler.inverse_transform(next_day))
    
    return future_predictions

In [ ]:
no_of_days = 10
future_results = predict_future(no_of_days, last_100)
print(future_results)

In [ ]:
future_results = np.array(future_results).reshape(-1,1)
plt.figure()
pd.DataFrame(future_results).plot(figsize=(15,5), marker='o')
for i in range(len(future_results)):
    plt.text(i,future_results[i],int(future_results[i][0]))
plt.xlabel('Future days')
plt.ylabel('Close price')
plt.title("Future Close price of bitcoin")

In [ ]:
model.save("bitcoin_model.keras")